In [ ]:
from os import mkdir, path
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cd /content/drive/MyDrive/Colab\ Notebooks/
!git clone https://github.com/SoonbeomChoi/miniSynth.git
%cd miniSynth
!git pull origin main

In [ ]:
!pip install pretty_midi

In [ ]:
# Change following paths before you run
DATASET_PATH = "/media/daftpunk3/home/soonbeom/Dataset/URMPPlus/edm_violin"
MIDI_FILE = "/media/daftpunk3/home/soonbeom/Dataset/URMPPlus/edm_violin/mid/BEVFE_19_Violin_80_BPM_G.mid"

In [ ]:
import torch
import torch.nn as nn
import librosa.display
import matplotlib.pyplot as plt
import IPython.display as ipd
from tqdm import tqdm

import config, preprocess, data, test
from model import Model

In [ ]:
# Run preprocess
preprocess.run(DATASET_PATH)

In [ ]:
# Train model
if not path.exists(config.exp_path):
    mkdir(config.exp_path)

dataloader = data.load()
model = Model().cuda()
optimizer = torch.optim.Adam(model.parameters(), config.learning_rate)
criterion = nn.L1Loss()
steps = 0

for epoch in range(int(config.stop_step/config.save_step)):
    model.train()
    progress_bar = tqdm(range(config.save_step), position=0, leave=True, total=None)
    for _ in progress_bar:
        note, mel = next(dataloader['train'])
        note = note.cuda()
        mel = mel.cuda()
        
        optimizer.zero_grad()
        mel_gen = model(note)
        loss = criterion(mel_gen, mel)
        loss.backward()
        optimizer.step()
        progress_bar.set_description(f"Loss - {loss.item():.4f}")
        steps += 1
        
    torch.save(model.state_dict(), path.join(config.exp_path, 'checkpoint.pt'))
    test.run(MIDI_FILE, savename=path.basename(MIDI_FILE).replace('.mid', f'_{steps}'))
    print(f"| steps: {steps} | train loss: {loss.item():.4f} |")

In [ ]:
audio = test.run(MIDI_FILE)
plt.figure(figsize=(14, 4))
librosa.display.waveplot(audio, sr=config.sample_rate)

In [ ]:
ipd.Audio(audio, rate=config.sample_rate)